# Summary of images processed

In [ ]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
sys.path.append(r"../LUCinSA_helpers")
from file_checks import *

In [ ]:
'''
PARAMETERS: modify in Notebook_settings notebook, then run that notebook and this cell to update here
DO not modify this cell
'''

%store -r basic_config
print("Basic Parameters: \n raw_dir = {} \n smooth_dir = {} \n local_dir = {} \n today = {}"
      .format(basic_config['raw_dir'],basic_config['smooth_dir'],basic_config['local_dir'],basic_config['today']))

In [ ]:
## To get all files processed in brdf directory across all processed cells:
all_files = pd.read_csv(Path('/home/downspout-cel/paraguay_lc/ALLFileList.csv'))
all_files.groupby(['yr','sensor']).size().unstack().plot(kind='bar', stacked=True, figsize=(20, 5),
            title=('Images processed for {}'.format(basic_config['country'])));

In [ ]:
used = all_files[all_files['quality']=='image_used']
not_used = all_files[all_files['quality']=='low_quality']
print('of the {} images ingested, {} were used in the final map product and {} were excluded due to quality issues'.format(len(used)+len(not_used),len(used),len(not_used)))
all_files.groupby(['yr','quality']).size().unstack().plot(color=['black','white'], kind='bar', stacked=True, edgecolor = 'black', figsize=(20, 5),
            title=('Image processing results for {}'.format(basic_config['country'])));

### To create / refersh list of processed images
Note this can take ~15 min

In [ ]:
#all_files = print_files_in_multiple_directories(basic_config['raw_dir'],"brdf",'.nc',print_list=True,out_dir=basic_config['local_dir'])

## To check all processed cells for missing files at download:
Note cell_processing_dl.csv is updated whenever check_log_files_dl.job is run
If this gets corrupted (e.g. if the script is run when there is no memory to save), it can be recreated by deleting the corrupted script and moving all of the dl logs from the arcihve folder to the run directory and running check_log_files_dl.job again. If multiple users have downloaded files, each must do this for it to be accounted for.

In [ ]:
dl_db_path = "/home/downspout-cel/paraguay_lc/cell_processing_dl.csv"
dl_db = pd.read_csv(Path(dl_db_path),index_col=[0])
dl_fix = dl_db[(dl_db['dl_fix_now']!='[]') & (pd.notnull(dl_db['dl_fix_now']))]
print(dl_fix)

## update full processing status db

In [ ]:
status_db_path = Path("/home/downspout-cel/paraguay_lc/cell_processing_post.csv")
update_cell_status_db(status_db_path, 'All', basic_config['raw_dir'], basic_config['smooth_dir'])

## Get full processing status

In [ ]:
post_db = pd.read_csv(Path(status_db_path),index_col=[0])
post_db.head(n=5)

## Get missing processing steps

In [ ]:
# cells without brdf processing yet (but with downloads)
no_brdf = post_db[post_db['num_brdf']!='brdf step not complete']
no_brdf.sort_index(inplace=True)
print('these cells are missing brdf files')
list(no_brdf.index)

In [ ]:
# cells without coreg processing yet (but with brdfs)
yes_brdf = post_db[post_db['num_brdf']=='brdf step not complete']
no_coreg = yes_brdf[yes_brdf['num_coreged']=='coreg step not complete']
no_coreg.sort_index(inplace=True)       
print('these cells have not completed coreg (but have brdfs)')
list(no_coreg.index)

## Get missing ts indices:

In [ ]:
ts_sum = [col for col in post_db.columns if 'index' in col]
print(ts_sum)

### Total number of scenes ingested
Note that a single Lansdat image is broken into ~80 grid cell images (A typical Landsat Scene = 31,000 km2 170 km x 185 km -- our grid cells are 400 km2), so images ingested needs to be divided by 80 to get an estimate of the number of actual Landsat / Sentinel scenes ingested. Method above takes a little longer (~15 min), but is much more precise.

In [ ]:
num_images_ingested_2022 = post_db['images_ingested_2022'].sum()
images_ingested = [col for col in post_db.columns if 'images_ingested' in col and 'All' not in col]
post_db['images_ingested_All'] = post_db[images_ingested].sum(axis=1)
num_images_ingested = post_db['images_ingested_All'].sum()
print('About {} images ingested in total for 2022 single-year product'.format(num_images_ingested_2022 // 80))
print('About {} images ingested in total 2000-2022 product'.format(num_images_ingested // 80))

### Total number of scenes used 
#### Excludes scenes not coregistered or not used for other data quality issues
Note comment about cells vs. images above. Method above takes a little longer (~15 min), but is much more precise.

In [ ]:
num_images_used_2022 = post_db['images_used_2022'].sum()
images_used = [col for col in post_db.columns if 'images_used' in col and 'All' not in col]
post_db['images_used_All'] = post_db[images_used].sum(axis=1)
num_images_used = post_db['images_used_All'].sum()
print('About {} images used in total for 2022 single-year product'.format(num_images_used_2022 // 80))
print('About {} images used in final map product for 2000-2022'.format(num_images_used // 80))

In [ ]:

all_files.groupby(['yr','sensor']).size().unstack().plot(kind='bar', stacked=True, figsize=(20, 5),
            title=('Images processed for {}'.format(basic_config['country'])));

## To save an html copy of this notebook with all outputs:

In [ ]:
### Run to print output as html

out_name = str(basic_config['country']+'5a_ImagesProcessed_'+basic_config['today'])
!jupyter nbconvert --output-dir='./Outputs' --to html --no-input --output=$out_name 5a_SummarizeData_ImagesProcessed.ipynb